# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [1]:
import pandas as pd
import datetime
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from Actions import GA_discrete_actions

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## Reading the Price data

In [2]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [4]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA Actions

In [ ]:
def evaluate_fitness(population, plant_params, prices):

    # To be written according to pumped storage optimisation problem
    fitness_scores = np.array([])

    for parameter_combination in population:

        water_level = plant_params["INITIAL_WATER_LEVEL"]
        fitness_score = 0

        for action, price in zip(parameter_combination, prices["spot"]):
            # Pump (-1)
            if action == -1:
                if water_level + plant_params["PUMP_RATE_M3H"] <= plant_params["MAX_STORAGE_M3"]:
                    fitness_score -= plant_params["PUMP_POWER_MW"] * price
                    water_level += plant_params["PUMP_RATE_M3H"]
                else:
                    fitness_score -= 10_000_000
            # Turbine (1)
            if action == 1:
                if water_level - plant_params["TURBINE_RATE_M3H"] >= plant_params["MIN_STORAGE_M3"]:
                    fitness_score += plant_params["TURBINE_POWER_MW"] * price
                    water_level -= plant_params["TURBINE_RATE_M3H"]
                else:
                    fitness_score -= 10_000_000
            # Do nothing (0)
            # Nothing happens to the fitness score and the water level
        
        fitness_scores = np.append(fitness_scores, fitness_score)

    return fitness_scores

In [ ]:
def trainable(config):
    ga = GA_discrete_actions(
        dna_size=config["WINDOW_SIZE"],
        discrete_action_space=[-1, 0, 1],
        elitism=config["ELITISM"],
        population_size=config["POPULATION"],
        mutation_rate=config["MUTATION_RATE"],
        survival_rate=config["SURVIVAL_RATE"],
        mutation_decay=config["MUTATION_DECAY"],
        mutation_limit=config["MUTATION_LIMIT"],
    )

    population = ga.initial_population
    average_fitnesses = []
    best_fitnesses = []
    generation = 0

    for _ in config["TOTAL_GENERATIONS"]:
        if (generation == 0) or (np.mean(average_fitnesses[-15:]) < ga.best_fitness * 0.99):
            population = ga.evolve(
                population=population,
                fitnesses=evaluate_fitness(
                    population=population, plant_params=plant_params, prices=example
                ),
            )
            average_fitnesses.append(ga.average_fitness)
            best_fitnesses.append(ga.best_fitness)

            if generation % 25 == 0:
                print(
                    f"Generation: {generation}, Best Fitness: {ga.best_fitness}, Average Fitness: {ga.average_fitness}, Mutation Rate: {ga.mutation_rate}"
                )
                # clear_output(wait=True)

            generation += 1

In [ ]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [ ]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": 200,
        "INITIAL_MUTATION_RATE": tune.choice(np.linspace(0.05, 0.5, 5)),
        "FINAL_MUTATION_RATE": tune.choice(np.linspace(0.01, 0.05, 5)),
        "INITIAL_EXPLORATION": 0.66,
        "ELITISM": tune.choice(np.linspace(0.05, 0.5, 10)),
    },
    total_generations=500,
    timeout_s=60*60*5,
)

In [ ]:
analysis.best_config

In [ ]:
pd.concat(analysis.trial_dataframes, axis=0).reset_index(drop=True).to_csv(f"./Tuning Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_GA_Elite.csv", index=False)

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
# (
#     analysis.dataframe()
#     .query("trial_id == 'd947e4c7'")
#     .filter(regex="config")
#     .melt()
#     .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
#     .set_index("variable")
#     .to_dict()["value"]
# )

### Tournament

In [ ]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [ ]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": 200,
        "CXPB": tune.uniform(0.05, 0.95),
        "INITIAL_MUTATION_RATE": np.linspace(0.05, 0.5, 5),
        "FINAL_MUTATION_RATE": np.linspace(0.01, 0.05, 5),
        "INITIAL_EXPLORATION": 0.66,
        "TOURNAMENT_SIZE": np.linspace(2, 100, 10),
    },
    total_generations=500,
    timeout_s=60*60*5,
)

In [ ]:
analysis.best_config

In [ ]:
pd.concat(analysis.trial_dataframes, axis=0).reset_index(drop=True).to_csv(f"./Tuning Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_GA_Tournament.csv", index=False)

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
(
    analysis.dataframe()
    .query("trial_id == 'dd531b71'")
    .filter(regex="config")
    .melt()
    .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
    .set_index("variable")
    .to_dict()["value"]
)